<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectifs" data-toc-modified-id="Objectifs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectifs</a></span></li><li><span><a href="#Dev" data-toc-modified-id="Dev-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dev</a></span></li><li><span><a href="#Industrialisation" data-toc-modified-id="Industrialisation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Industrialisation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Script" data-toc-modified-id="Script-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Script</a></span></li></ul></li></ul></li><li><span><a href="#Analyse-de-la-feature" data-toc-modified-id="Analyse-de-la-feature-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analyse de la feature</a></span><ul class="toc-item"><li><span><a href="#Stations-avec-le-plus-grand-nombre-de-non-prise-de-vélo-consécutif" data-toc-modified-id="Stations-avec-le-plus-grand-nombre-de-non-prise-de-vélo-consécutif-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Stations avec le plus grand nombre de non prise de vélo consécutif</a></span></li></ul></li></ul></div>

In [6]:
import pandas as pd

from vcub_keeper.reader.reader import *
from vcub_keeper.config import ROOT_DATA_CLEAN
from vcub_keeper.visualisation import plot_station_activity
from vcub_keeper.transform.features_factory import *

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 700

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Objectifs

- Permettre de savoir depuis combien de temps un vélo n'a pas été pris par station. Plus ce nombre est grand plus il y a de chance que la station soit HS. Cf https://github.com/armgilles/vcub_keeper/issues/10
- Industrialisation de la function.
- Analyse de la feature

## Dev

In [4]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = get_transactions_out(ts_activity)

In [5]:
ts_activity.dtypes

station_id                  uint64
date                datetime64[ns]
available_stands           float64
available_bikes            float64
status                     float64
transactions_in            float64
transactions_out           float64
transactions_all           float64
dtype: object

In [6]:
ts_activity['have_data'] = 1
ts_activity.loc[ts_activity['available_stands'].isna(),
                'have_data'] = 0

ts_activity['consecutive_no_transactions_out'] = \
    ts_activity.groupby(['station_id',
                        (ts_activity['have_data'] == 0).cumsum(),
                        (ts_activity['status'] == 0).cumsum(),
                        (ts_activity['transactions_out'] > 0).cumsum()]).cumcount()#['compteur'].cumsum() # 

ts_activity['consecutive_no_transactions_out'] = \
    ts_activity['consecutive_no_transactions_out'].fillna(0)

ts_activity.loc[ts_activity['available_stands'].isna(),
               'consecutive_no_transactions_out'] = 0

ts_activity = ts_activity.drop('have_data', axis=1)

In [7]:
ts_activity[(ts_activity['station_id'] == 1) & (ts_activity.date >= "2017-07-10 09:04:04")].head(15)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
0,1,2018-12-01 00:10:00,28.0,4.0,1.0,0.0,0.0,0.0,0
1,1,2018-12-01 00:20:00,28.0,4.0,1.0,0.0,0.0,0.0,1
2,1,2018-12-01 00:30:00,28.0,4.0,1.0,0.0,0.0,0.0,2
3,1,2018-12-01 00:40:00,27.0,5.0,1.0,1.0,0.0,1.0,3
4,1,2018-12-01 00:50:00,27.0,5.0,1.0,0.0,0.0,0.0,4
5,1,2018-12-01 01:00:00,28.0,4.0,1.0,0.0,1.0,1.0,0
6,1,2018-12-01 01:10:00,28.0,4.0,1.0,0.0,0.0,0.0,1
7,1,2018-12-01 01:20:00,28.0,4.0,1.0,0.0,0.0,0.0,2
8,1,2018-12-01 01:30:00,28.0,4.0,1.0,0.0,0.0,0.0,3
9,1,2018-12-01 01:40:00,29.0,3.0,1.0,0.0,1.0,1.0,0


Analyse des données entre 2 stations différentes

In [8]:
# Entre 2 stations différentes
ts_activity[91650:91660]

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
91650,1,2020-08-28 11:10:00,2.0,31.0,1.0,0.0,2.0,2.0,0
91651,1,2020-08-28 11:20:00,2.0,31.0,1.0,0.0,0.0,0.0,1
91652,1,2020-08-28 11:30:00,2.0,31.0,1.0,0.0,0.0,0.0,2
91653,1,2020-08-28 11:40:00,2.0,31.0,1.0,0.0,0.0,0.0,3
91654,1,2020-08-28 11:50:00,2.0,31.0,1.0,0.0,0.0,0.0,4
91655,2,2018-12-01 00:10:00,5.0,14.0,1.0,0.0,0.0,0.0,0
91656,2,2018-12-01 00:20:00,5.0,14.0,1.0,0.0,0.0,0.0,1
91657,2,2018-12-01 00:30:00,5.0,14.0,1.0,0.0,0.0,0.0,2
91658,2,2018-12-01 00:40:00,5.0,14.0,1.0,0.0,0.0,0.0,3
91659,2,2018-12-01 00:50:00,3.0,16.0,1.0,2.0,0.0,2.0,4


Analyse des données lorsqu'il manque des données.

In [9]:
station_id = 145
start_date = "2018-12-29"
end_date = "2019-01-03"

ts_activity[(ts_activity['station_id'] == station_id) &
            (ts_activity['date'] >= start_date) &
            (ts_activity['date'] <= end_date)]

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
13170474,145,2018-12-29 00:00:00,5.0,12.0,1.0,0.0,0.0,0.0,19
13170475,145,2018-12-29 00:10:00,5.0,12.0,1.0,0.0,0.0,0.0,20
13170476,145,2018-12-29 00:20:00,5.0,12.0,1.0,0.0,0.0,0.0,21
13170477,145,2018-12-29 00:30:00,NaN,NaN,0.0,0.0,NaN,0.0,0
13170478,145,2018-12-29 00:40:00,NaN,NaN,0.0,0.0,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...
13171190,145,2019-01-02 23:20:00,10.0,7.0,1.0,0.0,0.0,0.0,17
13171191,145,2019-01-02 23:30:00,10.0,7.0,1.0,0.0,0.0,0.0,18
13171192,145,2019-01-02 23:40:00,10.0,7.0,1.0,0.0,0.0,0.0,19
13171193,145,2019-01-02 23:50:00,11.0,6.0,1.0,0.0,1.0,1.0,0


In [10]:
ts_activity[(ts_activity['station_id'] == station_id) &
            (ts_activity['date'] >= "2019-01-02 11:30")].head(10)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
13171119,145,2019-01-02 11:30:00,NaN,NaN,0.0,0.0,NaN,0.0,0
13171120,145,2019-01-02 11:40:00,NaN,NaN,0.0,0.0,NaN,0.0,0
13171121,145,2019-01-02 11:50:00,14.0,3.0,1.0,0.0,0.0,9.0,1
13171122,145,2019-01-02 12:00:00,14.0,3.0,1.0,0.0,0.0,0.0,2
13171123,145,2019-01-02 12:10:00,13.0,4.0,1.0,1.0,0.0,1.0,3
13171124,145,2019-01-02 12:20:00,13.0,4.0,1.0,0.0,0.0,0.0,4
13171125,145,2019-01-02 12:30:00,13.0,4.0,1.0,0.0,0.0,0.0,5
13171126,145,2019-01-02 12:40:00,13.0,4.0,1.0,0.0,0.0,0.0,6
13171127,145,2019-01-02 12:50:00,12.0,5.0,1.0,1.0,0.0,1.0,7
13171128,145,2019-01-02 13:00:00,12.0,5.0,1.0,0.0,0.0,0.0,8


In [11]:
ts_activity[ts_activity['available_stands'].isna()].consecutive_no_transactions_out.unique()

array([0])

In [12]:
ts_activity[ts_activity['available_bikes'].isna()].consecutive_no_transactions_out.unique()

array([0])

Analyse lorsque qu'un station est déconnecté.

In [13]:
station_id = 172
ts_activity[(ts_activity['station_id'] == station_id) &
            (ts_activity['date'] >= "2020-05-11 06:30")].head(10)


,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
15717054,172,2020-05-11 06:30:00,28.0,0.0,0.0,0.0,0.0,0.0,0
15717055,172,2020-05-11 06:40:00,28.0,0.0,0.0,0.0,0.0,0.0,0
15717056,172,2020-05-11 06:50:00,28.0,0.0,1.0,0.0,0.0,0.0,1
15717057,172,2020-05-11 07:00:00,28.0,0.0,1.0,0.0,0.0,0.0,2
15717058,172,2020-05-11 07:10:00,24.0,4.0,1.0,4.0,0.0,4.0,3
15717059,172,2020-05-11 07:20:00,24.0,4.0,1.0,0.0,0.0,0.0,4
15717060,172,2020-05-11 07:30:00,24.0,4.0,1.0,0.0,0.0,0.0,5
15717061,172,2020-05-11 07:40:00,24.0,4.0,1.0,0.0,0.0,0.0,6
15717062,172,2020-05-11 07:50:00,24.0,4.0,1.0,0.0,0.0,0.0,7
15717063,172,2020-05-11 08:00:00,24.0,4.0,1.0,0.0,0.0,0.0,8


## Industrialisation

In [14]:
def get_consecutive_no_transactions_out(data):
    """
    Calcul depuis combien de temps la station n'a pas eu de prise de vélo. Plus le chiffre est haut, 
    plus ça fait longtemps que la station est inactive sur la prise de vélo.
    
    Si il n'y a pas de données d'activité pour la station (absence de 'available_stands'), 
    alors consecutive_no_transactions_out = 0 et une fois qu'il y a  a nouveau de l'activité (des données)
    le compteur `consecutive_no_transactions_out` reprend.

    Cet indicateur à aussi besoin que la station soit connecté (status == 1) afin de que le compteur
    avance (sinon = 0)
    L'indicateur pour s'activer et continer à avancer dans le temps doit avoir des vélos disposibles en
    station (plus de 2 vélos, possibilité vélo HS, borne HS...)
    
    Parameters
    ----------
    data : DataFrame
        Activité des stations Vcub avec la feature `transactions_out` (get_transactions_out)
    
    Returns
    -------
    data : DataFrame
        Ajout de colonne 'consecutive_no_transactions_out'
        
    Examples
    --------
    
    activite = get_consecutive_no_transactions_out(activite)
    """

    data['have_data'] = 1
    data.loc[data['available_stands'].isna(),
             'have_data'] = 0

    data['consecutive_no_transactions_out'] = \
        data.groupby(['station_id',
                      (data['available_bikes'] < 3).cumsum(), # 3 for 2 available_bikes
                      (data['have_data'] == 0).cumsum(),
                      (data['status'] == 0).cumsum(),
                      (data['transactions_out'] > 0).cumsum()]).cumcount()

    data['consecutive_no_transactions_out'] = \
        data['consecutive_no_transactions_out'].fillna(0)

    data.loc[data['available_stands'].isna(),
             'consecutive_no_transactions_out'] = 0

    data = data.drop('have_data', axis=1)
    
    return data

In [8]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = get_transactions_out(ts_activity)

/home/gillesa/github/vcub_keeper/src/vcub_keeper/reader/reader.py:122: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [16]:
ts_activity = get_consecutive_no_transactions_out(ts_activity)

In [17]:
# On regarde si la fonction prend en compte le nombre de Vcub dispo pour 
# avancer dans consecutive_no_transactions_out
#new = ts_activity[(ts_activity['station_id'] == 22) & 
#                  (ts_activity['date'] >= "2019-08-14 08:00:00")].head(60)

new = ts_activity[(ts_activity['station_id'] == 22) & 
                  (ts_activity['date'] >= "2019-11-28 00:00:00")].head(60)

new

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
1945005,22,2019-11-28 00:00:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945006,22,2019-11-28 00:10:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945007,22,2019-11-28 00:20:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945008,22,2019-11-28 00:30:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945009,22,2019-11-28 00:40:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945010,22,2019-11-28 00:50:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945011,22,2019-11-28 01:00:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945012,22,2019-11-28 01:10:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945013,22,2019-11-28 01:20:00,31.0,2.0,1.0,0.0,0.0,0.0,0
1945014,22,2019-11-28 01:30:00,31.0,2.0,1.0,0.0,0.0,0.0,0


#### Script 

In [1]:
from vcub_keeper.reader.reader import *
from vcub_keeper.transform.features_factory import *

In [9]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = get_transactions_out(ts_activity)
ts_activity = get_consecutive_no_transactions_out(ts_activity)

/home/gillesa/github/vcub_keeper/src/vcub_keeper/reader/reader.py:122: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [10]:
ts_activity.tail(10)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
16564502,251,2020-08-28 10:20:00,28.0,12.0,1.0,0.0,1.0,1.0,0
16564503,251,2020-08-28 10:30:00,28.0,12.0,1.0,0.0,0.0,0.0,1
16564504,251,2020-08-28 10:40:00,28.0,12.0,1.0,0.0,0.0,0.0,2
16564505,251,2020-08-28 10:50:00,28.0,12.0,1.0,0.0,0.0,0.0,3
16564506,251,2020-08-28 11:00:00,28.0,12.0,1.0,0.0,0.0,0.0,4
16564507,251,2020-08-28 11:10:00,28.0,12.0,1.0,0.0,0.0,0.0,5
16564508,251,2020-08-28 11:20:00,28.0,12.0,1.0,0.0,0.0,0.0,6
16564509,251,2020-08-28 11:30:00,26.0,14.0,1.0,2.0,0.0,2.0,7
16564510,251,2020-08-28 11:40:00,26.0,14.0,1.0,0.0,0.0,0.0,8
16564511,251,2020-08-28 11:50:00,26.0,14.0,1.0,0.0,0.0,0.0,9


## Analyse de la feature

In [21]:
ts_activity = read_time_serie_activity()

ts_activity = get_transactions_out(ts_activity)
ts_activity = get_consecutive_no_transactions_out(ts_activity)

TypeError: read_time_serie_activity() missing 1 required positional argument: 'path_directory'

### Stations avec le plus grand nombre de non prise de vélo consécutif

In [22]:
grp_station = ts_activity.groupby('station_id', as_index=False)['consecutive_no_transactions_out'].max()
grp_station = grp_station.sort_values('consecutive_no_transactions_out', ascending=0)

In [23]:
grp_station.head(10)

,station_id,consecutive_no_transactions_out
160,161,3431
159,160,3088
91,92,1742
79,80,1727
149,150,1639
70,71,1551
166,167,1527
179,180,1482
167,168,1331
181,182,1312


**Certaines stations sont très inactives (travaux ?)**

In [24]:
station_id = 172
start_date = "2020-05-03"
end_date = "2020-05-21"
tt = plot_station_activity(ts_activity, station_id=station_id, 
                      features_to_plot=['available_bikes', 'available_stands',
                                       'consecutive_no_transactions_out', 'status'],
                      start_date=start_date,
                      end_date=end_date,
                      return_data=True
                     )

# Test

In [11]:
from vcub_keeper.reader.reader import read_time_serie_activity
from vcub_keeper.transform.features_factory import get_transactions_out, get_consecutive_no_transactions_out

In [12]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = get_transactions_out(ts_activity)
ts_activity = get_consecutive_no_transactions_out(ts_activity)

/home/gillesa/github/vcub_keeper/src/vcub_keeper/reader/reader.py:122: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [17]:
ts_activity.dtypes

station_id                                 uint64
date                               datetime64[ns]
available_stands                          float64
available_bikes                           float64
status                                    float64
transactions_in                           float64
transactions_out                          float64
transactions_all                          float64
consecutive_no_transactions_out             int64
dtype: object

In [13]:
ts_activity.head()

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
0,1,2018-12-01 00:10:00,28.0,4.0,1.0,0.0,0.0,0.0,0
1,1,2018-12-01 00:20:00,28.0,4.0,1.0,0.0,0.0,0.0,1
2,1,2018-12-01 00:30:00,28.0,4.0,1.0,0.0,0.0,0.0,2
3,1,2018-12-01 00:40:00,27.0,5.0,1.0,1.0,0.0,1.0,3
4,1,2018-12-01 00:50:00,27.0,5.0,1.0,0.0,0.0,0.0,4


In [14]:
ts_activity = get_transactions_out(ts_activity)

In [15]:
ts_activity.head(8)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
0,1,2018-12-01 00:10:00,28.0,4.0,1.0,0.0,0.0,0.0,0
1,1,2018-12-01 00:20:00,28.0,4.0,1.0,0.0,0.0,0.0,1
2,1,2018-12-01 00:30:00,28.0,4.0,1.0,0.0,0.0,0.0,2
3,1,2018-12-01 00:40:00,27.0,5.0,1.0,1.0,0.0,1.0,3
4,1,2018-12-01 00:50:00,27.0,5.0,1.0,0.0,0.0,0.0,4
5,1,2018-12-01 01:00:00,28.0,4.0,1.0,0.0,1.0,1.0,0
6,1,2018-12-01 01:10:00,28.0,4.0,1.0,0.0,0.0,0.0,1
7,1,2018-12-01 01:20:00,28.0,4.0,1.0,0.0,0.0,0.0,2


In [32]:
# ts_activity[(ts_activity["transactions_out"] > 1) & (ts_activity["date"] == "2017-07-09 11:09:04")].head()
ts_activity[(ts_activity["date"] >= "2017-07-09 11:04:04") & (ts_activity["station_id"] == 1)].head(11).to_dict(orient="list")
# ts_activity[(ts_activity["date"] >= "2017-07-09 00:54:05") & (ts_activity["station_id"] == 22)].head(11).to_dict(orient="list")

{'station_id': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'date': [Timestamp('2018-12-01 00:10:00'),
  Timestamp('2018-12-01 00:20:00'),
  Timestamp('2018-12-01 00:30:00'),
  Timestamp('2018-12-01 00:40:00'),
  Timestamp('2018-12-01 00:50:00'),
  Timestamp('2018-12-01 01:00:00'),
  Timestamp('2018-12-01 01:10:00'),
  Timestamp('2018-12-01 01:20:00'),
  Timestamp('2018-12-01 01:30:00'),
  Timestamp('2018-12-01 01:40:00'),
  Timestamp('2018-12-01 01:50:00')],
 'available_stands': [28.0,
  28.0,
  28.0,
  27.0,
  27.0,
  28.0,
  28.0,
  28.0,
  28.0,
  29.0,
  29.0],
 'available_bikes': [4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0],
 'status': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'transactions_in': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'transactions_out': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 'transactions_all': [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 'consecutive_no_transactions_out': [0, 1, 2, 3, 4, 0, 1, 

In [ ]:
{'station_id': [22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22],
 'date': [Timestamp('2018-12-01 00:10:00'),
  Timestamp('2018-12-01 00:20:00'),
  Timestamp('2018-12-01 00:30:00'),
  Timestamp('2018-12-01 00:40:00'),
  Timestamp('2018-12-01 00:50:00'),
  Timestamp('2018-12-01 01:00:00'),
  Timestamp('2018-12-01 01:10:00'),
  Timestamp('2018-12-01 01:20:00'),
  Timestamp('2018-12-01 01:30:00'),
  Timestamp('2018-12-01 01:40:00'),
  Timestamp('2018-12-01 01:50:00')],
 'available_stands': [2.0,
  6.0,
  9.0,
  9.0,
  11.0,
  13.0,
  13.0,
  13.0,
  17.0,
  18.0,
  20.0],
 'available_bikes': [31.0,
  27.0,
  24.0,
  24.0,
  22.0,
  20.0,
  20.0,
  20.0,
  16.0,
  15.0,
  13.0],
 'status': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'transactions_in': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 'transactions_out': [0.0, 4.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0, 4.0, 1.0, 2.0],
 'transactions_all': [0.0, 4.0, 3.0, 0.0, 2.0, 2.0, 2.0, 0.0, 4.0, 1.0, 2.0],
 'consecutive_no_transactions_out': [0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0]}

In [33]:
data = {
    "station_id" : [1] * 11 + [22] * 11,
    "date" : [pd.Timestamp('2018-12-01 00:10:00'),
  pd.Timestamp('2018-12-01 00:20:00'),
  pd.Timestamp('2018-12-01 00:30:00'),
  pd.Timestamp('2018-12-01 00:40:00'),
  pd.Timestamp('2018-12-01 00:50:00'),
  pd.Timestamp('2018-12-01 01:00:00'),
  pd.Timestamp('2018-12-01 01:10:00'),
  pd.Timestamp('2018-12-01 01:20:00'),
  pd.Timestamp('2018-12-01 01:30:00'),
  pd.Timestamp('2018-12-01 01:40:00'),
  pd.Timestamp('2018-12-01 01:50:00')] +
        [pd.Timestamp('2018-12-01 00:10:00'),
  pd.Timestamp('2018-12-01 00:20:00'),
  pd.Timestamp('2018-12-01 00:30:00'),
  pd.Timestamp('2018-12-01 00:40:00'),
  pd.Timestamp('2018-12-01 00:50:00'),
  pd.Timestamp('2018-12-01 01:00:00'),
  pd.Timestamp('2018-12-01 01:10:00'),
  pd.Timestamp('2018-12-01 01:20:00'),
  pd.Timestamp('2018-12-01 01:30:00'),
  pd.Timestamp('2018-12-01 01:40:00'),
  pd.Timestamp('2018-12-01 01:50:00')],
    "available_stands" : [28.0, 28.0, 28.0, 27.0, 27.0, 28.0, 28.0, 28.0, 28.0, 29.0, 29.0] +
        [2.0, 6.0, 9.0, 9.0, 11.0, 13.0, 13.0, 13.0, 17.0, 18.0, 20.0],
    "available_bikes" : [4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0] +
        [31.0, 27.0, 24.0, 24.0, 22.0, 20.0, 20.0, 20.0, 16.0, 15.0, 13.0],
    "status" : [1] * 11 + [1] * 11,
    "transactions_out" : [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0] +
        [0.0, 4.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0, 4.0, 1.0, 2.0],
    'consecutive_no_transactions_out':[0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1] +
        [0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0], 
}

df_activite = pd.DataFrame(data)
# drop columns we want to test.
df_activite = df_activite.drop(columns=["transactions_out", "consecutive_no_transactions_out"], axis=1)

result = get_transactions_out(df_activite)
result = get_consecutive_no_transactions_out(result)

expected = pd.DataFrame(data)

pd.testing.assert_frame_equal(result, expected)

In [34]:
result

,station_id,date,available_stands,available_bikes,status,transactions_out,consecutive_no_transactions_out
0,1,2018-12-01 00:10:00,28.0,4.0,1,0.0,0
1,1,2018-12-01 00:20:00,28.0,4.0,1,0.0,1
2,1,2018-12-01 00:30:00,28.0,4.0,1,0.0,2
3,1,2018-12-01 00:40:00,27.0,5.0,1,0.0,3
4,1,2018-12-01 00:50:00,27.0,5.0,1,0.0,4
5,1,2018-12-01 01:00:00,28.0,4.0,1,1.0,0
6,1,2018-12-01 01:10:00,28.0,4.0,1,0.0,1
7,1,2018-12-01 01:20:00,28.0,4.0,1,0.0,2
8,1,2018-12-01 01:30:00,28.0,4.0,1,0.0,3
9,1,2018-12-01 01:40:00,29.0,3.0,1,1.0,0
